# Congestion Charging - Easy

![rel](https://sqlzoo.net/w/images/f/f6/CongestionCharge.png)

camera(**id**, perim)

keeper(**id**, name, address)

vehicle(**id**, keeper)

image(**_camera_**, **whn**, reg)

permit(**_reg_**, **sDate**, chargeType)

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@192.168.31.31:15432/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 60)


 ········


In [2]:
camera = pd.read_sql_table('camera', engine)
keeper = pd.read_sql_table('keeper', engine)
vehicle = pd.read_sql_table('vehicle', engine)
image = pd.read_sql_table('image', engine)
permit = pd.read_sql_table('permit', engine)

## Sample query

List the vehicles for which 'Strenuous, Sam' is the registered keeper. The link between Keepers and Vehicles is via the foreign key specified in the CREATE TABLE vehicle statement. Note the line:

```
 ,FOREIGN KEY(keeper) REFERENCES keeper(id)
```

This will be the basis of our join condition.

## 1.
Show the name and address of the keeper of vehicle SO 02 PSP.

In [3]:
(keeper.merge(vehicle.loc[vehicle['id']=='SO 02 PSP'],
              left_on='id', right_on='keeper')
 [['name', 'address']])

,name,address
0,"Strenuous, Sam",Surjection Street


## 2.
Show the number of cameras that take images for incoming vehicles.

In [4]:
pd.DataFrame({'cnt': [camera.loc[camera['perim']=='IN', 'id'].count()]})

,cnt
0,8


## 3.
List the image details taken by Camera 10 before 26 Feb 2007.

In [5]:
(image.loc[image['whn'] < '2007-02-26']
 .merge(camera, left_on='camera', right_on='id'))

,camera,whn,reg,id,perim
0,1,2007-02-25 06:10:13,SO 02 ASP,1,IN
1,9,2007-02-25 06:26:04,SO 02 ASP,9,OUT
2,9,2007-02-25 18:54:30,SO 02 DSP,9,OUT
3,9,2007-02-25 16:45:04,SO 02 HSP,9,OUT
4,9,2007-02-25 16:48:11,SO 02 HSP,9,OUT
5,9,2007-02-25 16:51:30,SO 02 HSP,9,OUT
6,9,2007-02-25 16:58:01,SO 02 ISP,9,OUT
7,9,2007-02-25 16:39:10,SO 02 RSP,9,OUT
8,9,2007-02-25 16:31:01,SO 02 SSP,9,OUT
9,17,2007-02-25 06:20:01,SO 02 ASP,17,None


## 4.
List the number of images taken by each camera. Your answer should show how many images have been taken by camera 1, camera 2 etc. The list must NOT include the images taken by camera 15, 16, 17, 18 and 19.

In [6]:
(image.loc[~ image['camera'].between(15, 19)]
 .groupby('camera').agg(cnt=pd.NamedAgg(column='reg', aggfunc='count'))
 .reset_index())

,camera,cnt
0,1,1
1,2,1
2,3,5
3,5,1
4,8,2
5,9,8
6,10,4
7,11,3
8,12,4


## 5.
A number of vehicles have permits that start on 30th Jan 2007. List the name and address for each keeper in alphabetical order without duplication.

In [7]:
(keeper.rename(columns={'id': 'keeper'})
 .merge(vehicle, on='keeper')
 .merge(permit.loc[permit['sdate']=='2007-01-30'],
        left_on='id', right_on='reg')
 [['name', 'address']].drop_duplicates()
 .sort_values('name'))


,name,address
0,"Ambiguous, Arthur",Absorption Ave.
4,"Assiduous, Annie",Attribution Alley
1,"Contiguous, Carol",Circumscription Close
3,"Strenuous, Sam",Surjection Street
